In [1]:
!git clone https://github.com/whyamistudyingcs/FYP_smoothing.git

Cloning into 'FYP_smoothing'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 51 (delta 15), reused 12 (delta 3), pack-reused 0
Unpacking objects: 100% (51/51), 16.42 KiB | 1.37 MiB/s, done.


In [2]:
!pip3 install textattack[tensorflow]
!pip3 install boto3 aiobotocore

  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - \ done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.

In [3]:
import torch
args = {
    'exp_dir': "./experiments/cls/",
    'exp_msg': "CLS Transformer",
    'gpu_idx': 10,

    'eval': True, #
    'model_dir_path': '/kaggle/input/imdb-bert-5', #
    'save_model': 'cls_trans', #
    'load_model': 'cls_trans_4', #
    'save': True, #

    'seed': 42, # 
    'device': "cuda" if torch.cuda.is_available() else "cpu", #

    'w_gn': 1.0,                    # embedding noise inference
    'noise_eps': 0.2,               # noise inference
    'single_layer': False,  #
    'nth_layers': 2,                # interval of injecting noise

    'num_ensemble': 10, # k_0
    'binom_ensemble': 50, # k_1
    'pooler_output': False, #
    'custom_forward': True,         # noise forwarding
    'binom_n_eval': 5,

    'sample_mask': False,
    'mask_batch_ratio': 1.0,        # percentage of masking , max is multi mask
    'rand_mask': False,
    'grad_mask_sample': False,
    'ens_grad_mask': 'rand',
    'two_step': False,
    'multi_mask': 2,                # number of masks
    'mask_idx': 103, #
    'vote_type': 'avg',

    'epochs': 5,
    'batch_size': 16, #
    'model': 'bert',                # model
    'optim': 'adamw',
    'scheduler': 'linear',
    'lr': 0.00001,
    'dropout': 0.1,
    'clip': 1.0,                    # clip gradient 
    'margin': 0.5,

    'embed_dim': 768,

    'dataset': 'imdb',#
    'num_classes': 2, #
    'pad_idx': 0,
    'max_seq_length': 256, #

    'binom_p': False, #
    'alpha_p': 0.70 #
}

attk_args = {
    'max_rate': 1,
    'dataset_type': 'test',
    'n_trials': 60,
    'shuffle': False,
    'save_data': True, # keep adversarial examples
    'attack_method': 'pwws',
    'adv_batch_size':16,
    'max_candidates': 48,
}

args = args | attk_args

## CustomWrapper

In [4]:
import torch
import transformers
import textattack

from textattack.models.wrappers.pytorch_model_wrapper import PyTorchModelWrapper
from textattack.attack_results import SuccessfulAttackResult, SkippedAttackResult, FailedAttackResult
from textattack.attack_results.attack_result import AttackResult

import pdb
import numpy as np

class AttackSummary:
    def __init__(self, max_seq_length=256):
        self._succeed = 0
        self._fail = 0
        self._skipped = 0
        
        self.pert_ratio = []
        self.pert_ratio_fail = []
        self.fail_q = []
        self.succ_q = []
        
        self.max_seq_length = max_seq_length
        
    def __str__(self):
        return ' || '.join(['{}: {:.2f}'.format(key, value) for (key, value) in self.get_metric().items()])
    
    def __call__(self, result):
        assert isinstance(result, AttackResult)
        if isinstance(result, SuccessfulAttackResult):
            self._succeed += 1

            n_query, n_pert, n_words = self.text_analysis(result)
            pert_word_ratio =(n_pert/n_words)*100
            self.pert_ratio.append(pert_word_ratio)

            self.succ_q.append(n_query)

        elif isinstance(result, FailedAttackResult):
            self._fail += 1
            n_query = result.num_queries
            self.fail_q.append(n_query)

            n_query, n_pert, n_words = self.text_analysis(result)
            pert_word_ratio =(n_pert/n_words)*100
            self.pert_ratio_fail.append(pert_word_ratio)

        elif isinstance(result, SkippedAttackResult):
            self._skipped += 1

    def text_analysis(self, result):
        words_pert = result.original_result.attacked_text.all_words_diff(result.perturbed_result.attacked_text)
        n_pert = len(words_pert)
        n_words = len(result.original_result.attacked_text.words)
        n_query = result.num_queries

        return n_query, n_pert, n_words

    def get_metric(self, reset: bool = False):
        all_numbers = self._succeed + self._fail + self._skipped
        correct_numbers = self._succeed + self._fail

        if correct_numbers == 0:
            success_rate = 0.0
        else:
            success_rate = self._succeed / correct_numbers
        
        if all_numbers == 0:
            clean_accuracy = 0.0
            robust_accuracy = 0.0
        else:
            clean_accuracy = correct_numbers / all_numbers
            robust_accuracy = self._fail / all_numbers

        try:
            pert_ratio = np.mean(self.pert_ratio)
        except:
            pert_ratio=0

        try:
            pert_ratio_fail = np.mean(self.pert_ratio_fail)
        except:
            pert_ratio_fail=0

        if len(self.succ_q)==0:
            avg_succ_q = 0
        else:
            avg_succ_q = np.array(self.succ_q).mean()
        if len(self.fail_q)==0:
            avg_fail_q = 0
        else:
            avg_fail_q = np.array(self.fail_q).mean()

        avg_query = np.mean(self.succ_q+self.fail_q)
        
        if reset:
            self.reset()

        out_stat = {"C-Acc" : clean_accuracy*100, "R-Acc" : robust_accuracy*100, "ASR": success_rate*100, 
                "Pert": pert_ratio, "Avg-Q": avg_query, "AvgS-Q":avg_succ_q, "AvgF-Q":avg_fail_q} 

        return out_stat

    def reset(self) -> None:
        self._succeed += 1
        self._fail += 1
        self._skipped += 1

def print_fn(args, f_name, batch_idx, num_successes, num_failed, num_skipped):
    n_trials = num_successes+num_failed+num_skipped
    valid_try =num_successes+num_failed
    
    if valid_try == 0:
        asr = 0
    else:
        asr = num_successes / (num_successes+num_failed)
    
    print('='*100)
    print(f"Model: {args['load_model']} || FileName: {f_name} || Attack: {args['attack_method']}")
    if args['multi_mask']:
        print(f"Multi_Mask: {args['multi_mask']}")
    print(f"Trials: {n_trials} || # Success: {num_successes} || # Failed: {num_failed} || # Skipped: {num_skipped}") 
    print(f"ASR: {asr:.4f} || AAcc: {num_failed/(n_trials):.4f}") 
    print('='*100)

class CustomWrapper(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer, args):
        assert isinstance(
            tokenizer,
            (transformers.PreTrainedTokenizer, transformers.PreTrainedTokenizerFast),
        ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer
        self.args = args
        self.num_classes = args["num_classes"]

        self.ensemble = True if args["num_ensemble"]>1 else False
        self.num_ensemble = args["num_ensemble"]
        self.multi_mask = args["multi_mask"]
        self.mask_idx = args["mask_idx"]
        self.pad_idx = args["pad_idx"]
        self.max_seq_length = args["max_seq_length"]

        self.binom_ensemble = args["binom_ensemble"]
        self.two_step = args["two_step"]
        self.binom_n_eval = args["binom_n_eval"]
        
    def __call__(self, text_input_list):
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=self.max_seq_length,
            truncation=True,
            return_tensors="pt",
        )
        
        model_device = self.args["device"] 
        inputs_dict.to(model_device)
        
        input_ids = inputs_dict['input_ids']
        attention_mask = inputs_dict['attention_mask']

        indices, _ = self.model.grad_mask(input_ids, attention_mask, pred=None, mask_filter=True)
        self.model.zero_grad()           
        logits = self.model.two_step_ensemble(input_ids, attention_mask, indices, self.num_ensemble, self.binom_ensemble)
        
        return logits
    
    def get_grad(self, text_input):
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for This model has not been implemented yet."
            )

        self.model.eval()

        embedding_layer = self.model.get_word_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True
        
        emb_grads = []
        
        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()

        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )

        input_dict.to(model_device)
        input_ids = input_dict['input_ids']
        attention_mask = input_dict['attention_mask']

        predictions = self.model(input_ids, attention_mask)
        predictions = predictions['logits']
        
        try:
            labels = predictions.argmax(dim=1)
            output = self.model(input_ids, attention_mask, labels)
            loss = output['loss']
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        
        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output
    
    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw to /usr/share/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2024-04-14 19:51:28.539676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:51:28.539775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:51:28.674779: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/
!unzip /usr/share/nltk_data/corpora/omw.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [6]:
import sys
sys.path.append('/kaggle/working/FYP_smoothing')

import torch

import numpy as np
import random
import os
import pandas as pd

import time, datetime
from datetime import timedelta
from model.adv_model import *
from model.load_model import *

from utils.utils import *
from utils.dataloader import *


if args['seed']>0:
    SEED = args['seed']
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

f_name = "_"+str(args['load_model'])+"_"+ args['dataset_type'] +".csv"
adv_path = os.path.join('./data/' + args['dataset'] +'_' + args['attack_method'] + f_name)

print("Load Dataset...")
if args['dataset_type'] == 'test':
    _, test = text_dataset(args['dataset'], args)
    dataset = test.train_test_split(test_size=args['n_trials'], seed=0)['test']
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=args['shuffle'], num_workers=0)
else:
    train, _ = text_dataset(args['dataset'], args)
    dataset = test.train_test_split(test_size=args['n_trials'], seed=0)['test']
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=args['shuffle'], num_workers=0)

print(f"Adv Path: {adv_path}") 
print(f"Save Data: {args['save_data']}") 

print("----------------------------------")
print(f"Dataset Type: {args['dataset_type']}") 
print("----------------------------------")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = load_tokenizer(args)

args['pad_idx'] = tokenizer.pad_token_id
args['mask_idx'] = tokenizer.mask_token_id
args['cls_token'] = tokenizer.cls_token_id
args['sep_token'] = tokenizer.sep_token_id


model = noisy_forward_loader(args)
model = SeqClsWrapper(model, args)
print("RSMI is Loaded")

model = load_checkpoint(model, args['load_model'], args['model_dir_path'])
model.eval()
print("Checkpoint is loaded")

print("Wrapping attack custom wrapper...")
model_wrapper = CustomWrapper(model, tokenizer, args)
model_wrapper.model.to(device)

# Attack recipe
if args['attack_method'] == 'pwws':
    from textattack.attack_recipes import PWWSRen2019
    attack = PWWSRen2019.build(model_wrapper)
elif args['attack_method'] == 'textfooler':
    from textattack.attack_recipes import TextFoolerJin2019
    attack = TextFoolerJin2019.build(model_wrapper)
elif args['attack_method'] == 'ours':
    #TODO
    raise Exception("Not Implemented")
else:
    raise Exception("Not Implemented")
    
attack.goal_function.maximizable = False
attack.goal_function.batch_size = args['adv_batch_size']
print(attack)

num_successes = 0
num_skipped = 0
num_failed = 0

df_adv = pd.DataFrame()
attack_result = AttackSummary(args['max_seq_length'])

start_t_gen = time.perf_counter()

print("Start Attack...")
for batch_idx, batch in enumerate(dataloader):
    label = batch['label'].item() # Ground truth label
    orig = batch['text'][0] # Clen Text 

    text_len = len(orig.split(" "))

    if text_len>args['max_seq_length']:
        orig = " ".join(orig.split(" ")[:args['max_seq_length']]) # truncation
        text_len = args['max_seq_length']
    
    # set query limit
    q_limit = int(args['max_candidates']*text_len) 
    attack.goal_function.query_budget = q_limit
    
    result = attack.attack(orig, label)
    attack_result(result)
    n_query, n_pert, n_words = attack_result.text_analysis(result)
    
    pert = result.perturbed_text()
    pert_word_ratio =(n_pert/text_len)*100
    
    if isinstance(result, SuccessfulAttackResult):
        num_successes+=1
        result_type = 'Successful'
    elif isinstance(result, FailedAttackResult):
        num_failed+=1
        result_type = 'Failed'
    elif isinstance(result, SkippedAttackResult):
        num_skipped+=1
        result_type = 'Skipped'
    
    # log and store the attack record
    adv_dict = {'pert': pert, 'orig': orig, 'ground_truth_output': label, 
            'result_type': result_type, 'n_query': n_query, 'n_pert': n_pert, 'n_words': n_words}
    df_adv = pd.concat([df_adv, pd.DataFrame([adv_dict])], ignore_index=True)
    
    if batch_idx%10==0:
        print(result.__str__(color_method='ansi'))
        print_fn(args, f_name, batch_idx, num_successes, num_failed, num_skipped)
        print(attack_result.__str__())

eval_t = time.perf_counter()-start_t_gen
print_fn(args, f_name, batch_idx, num_successes, num_failed, num_skipped)
print(attack_result.__str__())
print(f"Total Elapsed Time: {timedelta(seconds=eval_t)}") 

if args['save_data'] == True:
    if not os.path.isdir('./data/'):
        os.makedirs('./data/')
    
    df_adv.to_csv(adv_path)
    print(f"Save data...: {adv_path}") 

Load Dataset...


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Trainset Size: 22500
Testset Size: 25000
Adv Path: ./data/imdb_pwws_cls_trans_4_test.csv
Save Data: True
----------------------------------
Dataset Type: test
----------------------------------
Load Tokenizer...


...

RSMI is Loaded
Load: cls_trans_4
Checkpoint is loaded
Wrapping attack custom wrapper...
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


textattack: Unknown if model of class <class 'model.adv_model.SeqClsWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)
Start Attack...
1 (97%) --> 0 (60%)

This movie was awesome. It's a documentary about how surfing influenced skateboarding in the early days. It has interviews with skaters such as Tony Hawk(my idol)=), and Stacy Peralta to name a couple. Dogtown is a so called "ghetto" part of California, where there used to be an amusement park that was torn down. People started riding alongside the dangerous ruins of the park. Soon, the Zepher Surf team was formed. That led to Skateboardings first real start in Dogtown. The Z boys were a team of ragtag teenagers who loved skateboarding and started the phenomenon known as vert skating. They started it by skating in drained swimming pools. That's just a bit of the story behind it. It e

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0 (63%) --> [38:5:240m[SKIPPED]

All those that identify this as a simple rip-off of 'Elephant' - are there no other comments that you can make towards the movie on its own merits as an individual film (regardless of its apparent similarities to other movies).<br /><br />All those that question the validity of the movie - in terms of its stereotypical characters (the obligatory gay, the jock, the disabled kid, etc) - I'm not sure how long it has been since you were at school - but regardless of how amateurish the acting may be - the happenings that go one are surprisingly close to what may actually happen.<br /><br />And all those that disregard the film as being so simple: just six teenagers with the regular teenage angst that pushes one over the edge... did anyone stop to think, and take notice that the girl who took her life wasn't actually one of those six!! As one comment points out - she had screen time of maybe 2min max (excluding the final scenes). I think the point of the fil